# Imports

In [2]:
# %pylab

import numpy
import matplotlib
from matplotlib import pylab, mlab, pyplot
from mpl_toolkits.mplot3d import Axes3D
np = numpy
plt = pyplot

from pylab import *
from numpy import *

In [3]:
import os
from os import listdir
from os.path import isfile, join

import tkinter as tk
from tkinter import filedialog
%matplotlib qt

from pathlib import Path

from scipy.signal import savgol_filter

In [5]:
# Fonctions persos de Jofre et Adrien

# %run function_jofre.ipy
%run ../../Functions/1.2/data_fct_Adrien_point_by_point

matplotlib.rcParams.update({'font.size': 21})

In [4]:
# Constantes de la physique
# ABSOLUMENT RECHARGER APRÈS AVOIR EXECUTÉ LES CASES D'IMPORT AU DESSUS

C_e = 1.602e-19        # Coulomb
kb = 1.38064852*1e-23  # Boltzman
m_Ca = 40.078*1.66054e-27 # masse Ca 40.078
m_GM = 1e6*1.66054e-27 # la masse de la GMol
eps0 = 8.854187*1e-12  # permittivité électrique du vide

r0 = 2.5e-3 # 2.5e-3   # rayon piège Ca+
d0 = 4e-3/2            # longueur piège Ca+

Omega = 2.05e6*2*pi # 2.047e6

bk = 4 # nombre de barreaux par groupe (2 -> 4-pole , 4 -> 8-pole ...)

mkappa = 0.23          # écrantage piège réel GiantMol
wzLC = (2*pi*90806.9982303)**2
kappa_simion = m_Ca*d0**2*wzLC/(2*C_e)
print('%s = %f' % ('$\kappa_{simion}$',kappa_simion) )

zeta = kappa_simion*r0**2/d0**2

$\kappa_{simion}$ = 0.270471


# Selection fichiers a analyser

In [7]:
## GUI for data loading
# Select one data file all the way down to the directories
# SELECT Temp_SimuTypeQ_N ... .dat

file_cfg, slash_cfg, all_subdir = load_gui(filter_nocomplete=0)

file_path = file_cfg[0]
dir_path = file_cfg[1]
work_rep = file_cfg[2]
filename = file_cfg[3]

myslashpos = slash_cfg[0]
slashcond = slash_cfg[1]

address_runs = [x for x in all_subdir if 'DC01_RF08' in x]
print(address_runs)

> Répertoire : I:/Universite_Simus/20201110/DC01_RF08/Try00
> Filename : Temp_SimuTypeQ_N01024_Vrf0053_Udc0.1000D+01V_D1.0_S1.0RFG.dat
> myslashpos | [2, 19, 28, 38]
> slashcond | -2
No points deleted because they were not complete
Total number of data directories 84
['I:/Universite_Simus/20201110\\DC01_RF08\\Try00', 'I:/Universite_Simus/20201110\\DC01_RF08\\Try01', 'I:/Universite_Simus/20201110\\DC01_RF08\\Try02']


In [8]:
## Condition for data
points_and_coord, condition_parameters = simu_conditions(all_subdir, myslashpos, slashcond, filename)

['DC01_RF08', 'DC01_RF09', 'DC01_RF10', 'DC01_RF11', 'DC03_RF08', 'DC03_RF09', 'DC03_RF10', 'DC03_RF11', 'DC05_RF08', 'DC05_RF09', 'DC05_RF10', 'DC05_RF11', 'DC07_RF08', 'DC07_RF09', 'DC07_RF10', 'DC07_RF11', 'DC09_RF08', 'DC09_RF09', 'DC09_RF10', 'DC09_RF11', 'DC11_RF08', 'DC11_RF09', 'DC11_RF10', 'DC11_RF11', 'DC18_RF08', 'DC18_RF09', 'DC18_RF10', 'DC18_RF11']
000 > DC01_RF08
001 > DC01_RF09
002 > DC01_RF10
003 > DC01_RF11
004 > DC03_RF08
005 > DC03_RF09
006 > DC03_RF10
007 > DC03_RF11
008 > DC05_RF08
009 > DC05_RF09
010 > DC05_RF10
011 > DC05_RF11
012 > DC07_RF08
013 > DC07_RF09
014 > DC07_RF10
015 > DC07_RF11
016 > DC09_RF08
017 > DC09_RF09
018 > DC09_RF10
019 > DC09_RF11
020 > DC11_RF08
021 > DC11_RF09
022 > DC11_RF10
023 > DC11_RF11
024 > DC18_RF08
025 > DC18_RF09
026 > DC18_RF10
027 > DC18_RF11
> condition names ['DC', 'RF']
> number of points 28
> N_ions = 1024
> e_GMol = 50


# Fichiers init a recharger

In [9]:
## Data loading (1D arrays)
# select data with init to reuse
forcetryinit = ['Try00']

data_name, num_runs, PMandT, Gmol_data, r_LC_clip, dim_nu, all_subdir_reuseinit \
= data_retrieve_RFRelax(all_subdir,points_and_coord,condition_parameters,slash_cfg,
                                  forcecondnum = 1,
                                  forcetryinit = forcetryinit) # stopping_power

data0 = data_name[1]
data2 = data_name[2]
Tvar = PMandT[1]

> Points | 28
> Simulations pour chaque point | ['Try00', 'Try01', 'Try02']
(28, 3)


NameError: name 'all_subdir_aux' is not defined

In [8]:
# Automatic retrieve of GMol energy
aux_energie = file_path.find('0RFG')
aux_energie_unit = file_path.find('eV')
E0_GMol = file_path[aux_energie+4:aux_energie_unit+2]
E0_GMol = re.sub('k','000',E0_GMol)
E0_GMol = int(E0_GMol.strip('eV'))
print(E0_GMol)

1000


In [9]:
# Where is written Detuning and how much it is
temp_pos = filename.find('0RFG')
Detuning = filename[temp_pos-7:temp_pos-4]
print(Detuning)

1.0


In [15]:
## Data loading with hardcode names
eta = 1.5e-20

plot_point = 15
plot_try = 0
# str_eta_chiffre    = float(f'{eta[plot_point]:1.2e}'[:4])/10
# str_eta_puissance  = int(re.sub('e','D',f'{eta[plot_point]:1.2e}'[6:]))-1
# str_eta = f'{str_eta_chiffre:1.2f}'+'D-'+str(str_eta_puissance)
str_eta = '0.15D-19'
dirname_hc = str(data0[plot_point])+'/'
# Temp_3D_Harmo_N1024_T500uK_F0.20D-20Kg_s_5
datainit_hc = 'Temp_3D_Harmo_N1024_T500uK_F'+str(str_eta)+'Kg_s_5'
data0_hc    = '/Temp_SimuTypeQ_N01024_Vrf0070_Udc0.5000D+01V_D1.0_S1.0RFG'
data4_hc    = '/Temp_SimuType4_01_N01024_Vrf0070_Udc0.5000D+01V_D1.0_S1.0RFG'
data2_hc    = '/Temp_SimuType2_01_N01024_Vrf0070_Udc0.5000D+01V_D1.0_S1.0RFG'

print(dirname_hc)
print(datainit_hc)

['I:/Universite_Simus/20200903\\DC07_RF11\\Try00/Temp_SimuTypeQ_N01024_Vrf0070_Udc0.4000D+01V_D1.0_S1.0RFG']/
Temp_3D_Harmo_N1024_T500uK_F0.15D-19Kg_s_5


In [11]:
# recover duration for each step of simu (i_free__fly, i_laser_fly ...)
plot_try = 1
address = all_subdir_reuseinit[plot_point]
print(address)
info_files = sort( [f for f in listdir(address) if isfile(join(address, f)) and ".info" in f] )
print(sort(info_files))


tmp_info = loadtxt(all_subdir_reuseinit[plot_point]+'/'+info_files[0],encoding ='utf-8',
            skiprows=1, usecols=(0),
            unpack=True)
eta = tmp_info[-2]
i_free__fly_Lan = tmp_info[-1]

tmp_info = loadtxt(all_subdir[0]+'/'+'xva_SimuType6_01_N01024_Vrf0053_Udc0.1000D+01V_D1.0_S1.0RFG1keV.info',encoding ='utf-8',
            skiprows=0, usecols=(0),
            unpack=True)
i_laser_fly_quench = tmp_info[-2]
i_laser_fly = tmp_info[-1]

n_dt = 100
dt = 1/2e6

print('eta',eta)
print('i_free__fly_Lan',i_free__fly_Lan)
print('i_laser_fly_quench',i_laser_fly_quench)
print('i_laser_fly',i_laser_fly)

I:/Universite_Simus/20200903\DC07_RF11\Try00
['Langevin_cooling.info'
 'xva_SimuType2_01_N01024_Vrf0070_Udc0.4000D+01V_D1.0_S1.0RFG50eV.info'
 'xva_SimuType4_01_N01024_Vrf0070_Udc0.4000D+01V_D1.0_S1.0RFG50eV.info'
 'xva_SimuTypeQ_N01024_Vrf0070_Udc0.4000D+01V_D1.0_S1.0RFG.info']
eta 1.5e-20
i_free__fly_Lan 1000.0
i_laser_fly_quench 8000.0
i_laser_fly 110000.0


In [12]:
# Get T and fluo PM  All SimuType
address_reuseinit = all_subdir_reuseinit[plot_point]
address = all_subdir[plot_point]
init_data = data0[plot_point][0]
evol_data = data2[plot_point][0]
print(init_data)
print(evol_data)

# onlyfiles = [f for f in listdir(evol_data) if isfile(join(evol_data, f)) and "SimuType" in f and ".dat" in f]
# print(sort(onlyfiles))
# onlyfiles_reuseinit = [f for f in listdir(init_data) if isfile(join(init_data, f)) and "SimuType" in f and ".dat" in f]
# print(sort(onlyfiles_reuseinit))

ta, te, ti = plot_T_and_PM_InitQ_Evol_AfterCool(address_reuseinit,address,
                                           flag_plot=1,fig_name='0o0o0o0',xlim1=(0,65),ylim1=(2e-4,6000),ylim2=(-50,3300))

fname = address[29:38]
plt.figure(fname).clf()
ax1 = subplot(111)
te_filter = []
for k in range(3):
    te_filter.append(savgol_filter(te[:,k], 25, 1))
    ax1.semilogy(ta*1e3,te_filter[k])
    ax1.grid()
    
ax1.set_xlabel('time [µs]')
ax1.set_ylabel('T [K]')
title(fname)
    
i_free__fly_Lan = 1000
i_laser_fly = 6000
i_laser_fly_quench = 8000
i_relax_fly = 5000 * 22
    
time_switch = i_free__fly_Lan *dt
ax1.vlines(time_switch*1e3,1e-4,1e4,color='blue')
time_switch = (i_free__fly_Lan+i_laser_fly)*dt
ax1.vlines(time_switch*1e3,1e-4,1e4,color='green')
time_switch = (i_free__fly_Lan+i_laser_fly+i_laser_fly_quench)*dt
ax1.vlines(time_switch*1e3,1e-4,1e4,color='orange')
time_switch = (i_free__fly_Lan+i_laser_fly+i_laser_fly_quench+i_relax_fly)*dt
ax1.vlines((time_switch+ta[2]-ta[1])*1e3,1e-4,1e4,color='purple')

tight_layout()

# tt2, T_CM2, T_aux2, PM2
# ,xlim1=(3.458,3.582),ylim1=(2e-4,50),ylim2=(19.5,65)

I:/Universite_Simus/20200903\DC07_RF11\Try00/Temp_SimuTypeQ_N01024_Vrf0070_Udc0.4000D+01V_D1.0_S1.0RFG
I:/Universite_Simus/20201028\DC07_RF11\Try00/Temp_SimuType6_01_N01024_Vrf0070_Udc0.4000D+01V_D1.0_S1.0RFG1keV
len save_T [55440]
[5.05000000e-07 1.50946576e-06 3.24106192e-05 8.64160446e-09
 4.69168154e+01 1.56441196e+00 6.55206829e+01]


C:\Users\adr1e\Documents\GitHub\GiantMol\Functions\1.2\data_fct_Adrien_point_by_point_reuseinit.py:329: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  plt.tight_layout()
H:\Programmes\Anaconda3\envs\py307\lib\site-packages\ipykernel\eventloops.py:106: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  app.exec_()


In [13]:
my_order = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,24,25,26,27,20,21,22,23]

In [16]:
# Plot all temp for all conditions
pos_aux = all_subdir[0].find('DC')
my_date = all_subdir[0][pos_aux-9:pos_aux-1]
fname = 'Temp_allpoints_'+my_date+'_'+forcetryinit[0]
cm = pylab.get_cmap('rainbow')

fig, ax = plt.subplots(4, 7,num=fname,clear=True,sharex=True,sharey=True)  # (a,b)
fig.set_size_inches(11.69,8.27)
# suptitle('Saturation 866')

to_plot = 0

for l in range(0,7): # b
    for k in range(0,4): # a
        plot_point = my_order[to_plot]
#             str_eta_chiffre    = float(f'{eta[patate]:1.2e}'[:4])/10
#             str_eta_puissance  = int(re.sub('e','D',f'{eta[patate]:1.2e}'[6:]))-1
#             str_eta = f'{str_eta_chiffre:1.2f}'+'D-'+str(str_eta_puissance)
        str_eta = '0.15D-19'
        address_reuseinit = all_subdir_reuseinit[plot_point]
        address = all_subdir[plot_point]

        info_files = sort( [f for f in listdir(data0[plot_point][plot_try][:-58]) if isfile(join(data0[plot_point][plot_try][:-58], f)) and ".bin" and "3D" in f] )
        N_ions, j_save, dt_j_save_next, eta_float, Temp, save_T = \
        load_Temp_init_bin_Lan(data0[plot_point][plot_try][:-57]+info_files[0].strip('.bin'),flag_print=0)

        # Get T and fluo PM
        tt, T_aux, T_CM = plot_T_and_PM_InitQ_Evol_AfterCool(address_reuseinit,address,
                                       flag_plot=0,fig_name='0o0o0o0',xlim1=(0,65),ylim1=(2e-4,6000),ylim2=(-50,3300))


        tt_Lan = linspace(0,dt_j_save_next*j_save,j_save)
        tt_md = tt

#         tt_Lan = linspace(0,dt_j_save_next*j_save,j_save) # dt_j_save_next*100 duration of Langevin run  ,
#         tt_md = ta[0]
#         T_aux = te[0]
#         tt = concatenate((tt_Lan,tt_md))
#         T_all = concatenate((save_T[:,4:],T_aux),axis=0)

        tt = concatenate((tt_Lan,tt_md))
        T_all = concatenate((save_T[:,4:],T_aux),axis=0)

        ax[3-k,l].semilogy(tt*1e3,T_all[:,0], label='Tx',color='C0')
        ax[3-k,l].semilogy(tt*1e3,T_all[:,1], label='Ty',color='C1')
        ax[3-k,l].semilogy(tt*1e3,T_all[:,2], label='Tz',color='C2')
        ax[3-k,l].grid(which='both')
        ax[3-k,l].annotate(str(list(dict.fromkeys(points_and_coord))[plot_point]), xy=(0.20, 0.99), xycoords='axes fraction', # pts_concerned[to_plot]
            size=12, ha='left', va='top',
            bbox=dict(boxstyle='round', fc=None))
        to_plot += 1
        if to_plot == 28:
            break
    if to_plot == 28:
        break
ax[-1,-1].set_xlim(-1,72)
ax[-1,-1].set_ylim(2e-4,250000)

fig.suptitle(fname)
fig.text(0.56, 0.04, 'time [ms]', ha='center')
fig.text(0.04, 0.54, 'T [K]', va='center', rotation='vertical')

# fig.suptitle(pt_to_plot[0][:-6],fontsize=10)

# fig.text(0.5, 0.01, 'f', ha='center')
# fig.text(0.01, 0.5, r'$| FFT(HC(t)) | / N_{sample}$', va='center', rotation='vertical')

fig.set_size_inches(11.69,8.27)
plt.tight_layout()
subplots_adjust(hspace=0.05, wspace = 0.05, bottom = 0.120, left = 0.140, top = 0.92) # hspace=0, top=0.95, left = 0.065, bottom = 0.07
# savefig(fname+'.png') 

len save_T [55440]
[5.05000000e-07 1.41358910e-05 2.19387267e-03 1.11238728e-05
 4.52098528e+01 1.51182185e+00 7.29968464e+01]
len save_T [55440]
[5.05000000e-07 1.41358910e-05 2.19387267e-03 1.11238728e-05
 4.52098528e+01 1.51182185e+00 7.29968464e+01]
len save_T [55440]
[5.05000000e-07 1.18501210e-07 1.09167203e-02 4.54162497e-06
 4.32063724e+01 1.13421880e+00 6.95508793e+01]
len save_T [55440]
[5.05000000e-07 1.18501210e-07 1.09167203e-02 4.54162497e-06
 4.32063724e+01 1.13421880e+00 6.95508793e+01]
len save_T [55440]
[5.05000000e-07 1.75991917e-05 1.98133826e-03 2.13661821e-05
 4.31263955e+01 1.24852747e+00 6.91596241e+01]
len save_T [55440]
[5.05000000e-07 1.75991917e-05 1.98133826e-03 2.13661821e-05
 4.31263955e+01 1.24852747e+00 6.91596241e+01]
len save_T [55440]
[5.05000000e-07 1.91641179e-04 9.27131488e-05 3.40265653e-07
 5.61790996e+01 1.94840836e+00 7.47446665e+01]
len save_T [55440]
[5.05000000e-07 1.91641179e-04 9.27131488e-05 3.40265653e-07
 5.61790996e+01 1.94840836e+00 

In [17]:
# Plot all temp for all conditions
pos_aux = all_subdir[0].find('DC')
my_date = all_subdir[0][pos_aux-9:pos_aux-1]
fname = 'Temp_allpoints_zoom_'+my_date+'_'+forcetryinit[0]
cm = pylab.get_cmap('rainbow')

fig, ax = plt.subplots(4, 7,num=fname,clear=True,sharex=True,sharey=True)  # (a,b)
fig.set_size_inches(11.69,8.27)
# suptitle('Saturation 866')

to_plot = 0

for l in range(0,7): # b
    for k in range(0,4): # a
        plot_point = my_order[to_plot]
#             str_eta_chiffre    = float(f'{eta[patate]:1.2e}'[:4])/10
#             str_eta_puissance  = int(re.sub('e','D',f'{eta[patate]:1.2e}'[6:]))-1
#             str_eta = f'{str_eta_chiffre:1.2f}'+'D-'+str(str_eta_puissance)
        str_eta = '0.15D-19'
        address_reuseinit = all_subdir_reuseinit[plot_point]
        address = all_subdir[plot_point]

        info_files = sort( [f for f in listdir(data0[plot_point][plot_try][:-58]) if isfile(join(data0[plot_point][plot_try][:-58], f)) and ".bin" and "3D" in f] )
        N_ions, j_save, dt_j_save_next, eta_float, Temp, save_T = \
        load_Temp_init_bin_Lan(data0[plot_point][plot_try][:-57]+info_files[0].strip('.bin'),flag_print=0)

        # Get T and fluo PM
        tt, T_aux, T_CM = plot_T_and_PM_InitQ_Evol_AfterCool(address_reuseinit,address,
                                       flag_plot=0,fig_name='0o0o0o0',xlim1=(0,65),ylim1=(2e-4,6000),ylim2=(-50,3300))


        tt_Lan = linspace(0,dt_j_save_next*j_save,j_save)
        tt_md = tt

#         tt_Lan = linspace(0,dt_j_save_next*j_save,j_save) # dt_j_save_next*100 duration of Langevin run  ,
#         tt_md = ta[0]
#         T_aux = te[0]
#         tt = concatenate((tt_Lan,tt_md))
#         T_all = concatenate((save_T[:,4:],T_aux),axis=0)

        tt = concatenate((tt_Lan,tt_md))
        T_all = concatenate((save_T[:,4:],T_aux),axis=0)

        ax[3-k,l].semilogy(tt*1e3,T_all[:,0], label='Tx',color='C0')
        ax[3-k,l].semilogy(tt*1e3,T_all[:,1], label='Ty',color='C1')
        ax[3-k,l].semilogy(tt*1e3,T_all[:,2], label='Tz',color='C2')
        ax[3-k,l].grid(which='both')
        ax[3-k,l].annotate(str(list(dict.fromkeys(points_and_coord))[plot_point]), xy=(0.20, 0.99), xycoords='axes fraction', # pts_concerned[to_plot]
            size=12, ha='left', va='top',
            bbox=dict(boxstyle='round', fc=None))
        to_plot += 1
        if to_plot == 28:
            break
    if to_plot == 28:
        break
ax[-1,-1].set_xlim(-1,72)
ax[-1,-1].set_ylim(2e-4,5e-3)

fig.suptitle(fname)
fig.text(0.56, 0.04, 'time [ms]', ha='center')
fig.text(0.04, 0.54, 'T [K]', va='center', rotation='vertical')

# fig.suptitle(pt_to_plot[0][:-6],fontsize=10)

# fig.text(0.5, 0.01, 'f', ha='center')
# fig.text(0.01, 0.5, r'$| FFT(HC(t)) | / N_{sample}$', va='center', rotation='vertical')

fig.set_size_inches(11.69,8.27)
plt.tight_layout()
subplots_adjust(hspace=0.05, wspace = 0.05, bottom = 0.120, left = 0.140, top = 0.92) # hspace=0, top=0.95, left = 0.065, bottom = 0.07
savefig(fname+'.png') 

len save_T [55440]
[5.05000000e-07 1.41358910e-05 2.19387267e-03 1.11238728e-05
 4.52098528e+01 1.51182185e+00 7.29968464e+01]
len save_T [55440]
[5.05000000e-07 1.41358910e-05 2.19387267e-03 1.11238728e-05
 4.52098528e+01 1.51182185e+00 7.29968464e+01]
len save_T [55440]
[5.05000000e-07 1.18501210e-07 1.09167203e-02 4.54162497e-06
 4.32063724e+01 1.13421880e+00 6.95508793e+01]
len save_T [55440]
[5.05000000e-07 1.18501210e-07 1.09167203e-02 4.54162497e-06
 4.32063724e+01 1.13421880e+00 6.95508793e+01]
len save_T [55440]
[5.05000000e-07 1.75991917e-05 1.98133826e-03 2.13661821e-05
 4.31263955e+01 1.24852747e+00 6.91596241e+01]
len save_T [55440]
[5.05000000e-07 1.75991917e-05 1.98133826e-03 2.13661821e-05
 4.31263955e+01 1.24852747e+00 6.91596241e+01]
len save_T [55440]
[5.05000000e-07 1.91641179e-04 9.27131488e-05 3.40265653e-07
 5.61790996e+01 1.94840836e+00 7.47446665e+01]
len save_T [55440]
[5.05000000e-07 1.91641179e-04 9.27131488e-05 3.40265653e-07
 5.61790996e+01 1.94840836e+00 

In [37]:
# storing grad od savitzky of temperature
dT_filtered = np.zeros( (len(all_subdir), len(savgol_filter(T_all[:,0], 25, 1)), 3) )
for k,_ in enumerate(all_subdir):
    for l in range(3):
        
        address_reuseinit = all_subdir_reuseinit[k]
        address = all_subdir[k]

        info_files = sort( [f for f in listdir(data0[k][plot_try][:-58]) if isfile(join(data0[k][plot_try][:-58], f)) and ".bin" and "3D" in f] )
        N_ions, j_save, dt_j_save_next, eta_float, Temp, save_T = \
        load_Temp_init_bin_Lan(data0[k][plot_try][:-57]+info_files[0].strip('.bin'),flag_print=0)

        # Get T and fluo PM
        tt, T_aux, T_CM = plot_T_and_PM_InitQ_Evol_AfterCool(address_reuseinit,address,
                                       flag_plot=0,fig_name='0o0o0o0',xlim1=(0,65),ylim1=(2e-4,6000),ylim2=(-50,3300))

        tt_Lan = linspace(0,dt_j_save_next*j_save,j_save)
        tt = concatenate((tt_Lan,tt_md))
        T_all = concatenate((save_T[:,4:],T_aux),axis=0)
        
        dT_filtered[k,:,l] = gradient( savgol_filter(T_all[:,l], 25, 1) )

len save_T [55440]
[5.05000000e-07 1.41358910e-05 2.19387267e-03 1.11238728e-05
 4.52098528e+01 1.51182185e+00 7.29968464e+01]
len save_T [55440]
[5.05000000e-07 1.41358910e-05 2.19387267e-03 1.11238728e-05
 4.52098528e+01 1.51182185e+00 7.29968464e+01]
len save_T [55440]
[5.05000000e-07 1.41358910e-05 2.19387267e-03 1.11238728e-05
 4.52098528e+01 1.51182185e+00 7.29968464e+01]
len save_T [55440]
[5.05000000e-07 1.41358910e-05 2.19387267e-03 1.11238728e-05
 4.52098528e+01 1.51182185e+00 7.29968464e+01]
len save_T [55440]
[5.05000000e-07 1.41358910e-05 2.19387267e-03 1.11238728e-05
 4.52098528e+01 1.51182185e+00 7.29968464e+01]
len save_T [55440]
[5.05000000e-07 1.41358910e-05 2.19387267e-03 1.11238728e-05
 4.52098528e+01 1.51182185e+00 7.29968464e+01]
len save_T [55440]
[5.05000000e-07 1.18501210e-07 1.09167203e-02 4.54162497e-06
 4.32063724e+01 1.13421880e+00 6.95508793e+01]
len save_T [55440]
[5.05000000e-07 1.18501210e-07 1.09167203e-02 4.54162497e-06
 4.32063724e+01 1.13421880e+00 

In [36]:
dT_filtered[12,:,0]

array([-2.03371445, -2.03371445, -2.03371445, ...,  1.07793045,
        1.07793045,  1.07793045])

In [20]:
# Plot dERIVATIVE all temp for all conditions
pos_aux = all_subdir[0].find('DC')
my_date = all_subdir[0][pos_aux-9:pos_aux-1]
fname = 'dTemp_allpoints_zoom_'+my_date+'_'+forcetryinit[0]
cm = pylab.get_cmap('rainbow')

fig, ax = plt.subplots(4, 7,num=fname,clear=True,sharex=True,sharey=True)  # (a,b)
fig.set_size_inches(11.69,8.27)
# suptitle('Saturation 866')

to_plot = 0

for l in range(0,7): # b
    for k in range(0,4): # a
        plot_point = my_order[to_plot]
#             str_eta_chiffre    = float(f'{eta[patate]:1.2e}'[:4])/10
#             str_eta_puissance  = int(re.sub('e','D',f'{eta[patate]:1.2e}'[6:]))-1
#             str_eta = f'{str_eta_chiffre:1.2f}'+'D-'+str(str_eta_puissance)
        str_eta = '0.15D-19'
        address_reuseinit = all_subdir_reuseinit[plot_point]
        address = all_subdir[plot_point]

        info_files = sort( [f for f in listdir(data0[plot_point][plot_try][:-58]) if isfile(join(data0[plot_point][plot_try][:-58], f)) and ".bin" and "3D" in f] )
        N_ions, j_save, dt_j_save_next, eta_float, Temp, save_T = \
        load_Temp_init_bin_Lan(data0[plot_point][plot_try][:-57]+info_files[0].strip('.bin'),flag_print=0)

        # Get T and fluo PM
        tt, T_aux, T_CM = plot_T_and_PM_InitQ_Evol_AfterCool(address_reuseinit,address,
                                       flag_plot=0,fig_name='0o0o0o0',xlim1=(0,65),ylim1=(2e-4,6000),ylim2=(-50,3300))


        tt_Lan = linspace(0,dt_j_save_next*j_save,j_save)
        tt_md = tt

#         tt_Lan = linspace(0,dt_j_save_next*j_save,j_save) # dt_j_save_next*100 duration of Langevin run  ,
#         tt_md = ta[0]
#         T_aux = te[0]
#         tt = concatenate((tt_Lan,tt_md))
#         T_all = concatenate((save_T[:,4:],T_aux),axis=0)

        tt = concatenate((tt_Lan,tt_md))
        T_all = concatenate((save_T[:,4:],T_aux),axis=0)
        
        ax[3-k,l].plot(tt*1e3,gradient( savgol_filter(T_all[:,0], 25, 1) ), label='Tx',color='C0')
        ax[3-k,l].plot(tt*1e3,gradient( savgol_filter(T_all[:,1], 25, 1) ), label='Ty',color='C1')
        ax[3-k,l].plot(tt*1e3,gradient( savgol_filter(T_all[:,2], 25, 1) ), label='Tz',color='C2')
        ax[3-k,l].grid(which='both')
        ax[3-k,l].annotate(str(list(dict.fromkeys(points_and_coord))[plot_point]), xy=(0.20, 0.99), xycoords='axes fraction', # pts_concerned[to_plot]
            size=12, ha='left', va='top',
            bbox=dict(boxstyle='round', fc=None))
        to_plot += 1
        if to_plot == 28:
            break
    if to_plot == 28:
        break
# ax[-1,-1].set_xlim(-1,72)
ax[-1,-1].set_ylim(-1e-7,1e-4)

fig.suptitle(fname)
fig.text(0.56, 0.04, 'time [ms]', ha='center')
fig.text(0.04, 0.54, '$\delta$T/$\delta$t [K]', va='center', rotation='vertical')

# fig.suptitle(pt_to_plot[0][:-6],fontsize=10)

# fig.text(0.5, 0.01, 'f', ha='center')
# fig.text(0.01, 0.5, r'$| FFT(HC(t)) | / N_{sample}$', va='center', rotation='vertical')

fig.set_size_inches(11.69,8.27)
plt.tight_layout()
subplots_adjust(hspace=0.05, wspace = 0.05, bottom = 0.120, left = 0.140, top = 0.92) # hspace=0, top=0.95, left = 0.065, bottom = 0.07
# savefig(fname+'.png') 

len save_T [55440]
[5.05000000e-07 1.41358910e-05 2.19387267e-03 1.11238728e-05
 4.52098528e+01 1.51182185e+00 7.29968464e+01]
len save_T [55440]
[5.05000000e-07 1.41358910e-05 2.19387267e-03 1.11238728e-05
 4.52098528e+01 1.51182185e+00 7.29968464e+01]
len save_T [55440]
[5.05000000e-07 1.18501210e-07 1.09167203e-02 4.54162497e-06
 4.32063724e+01 1.13421880e+00 6.95508793e+01]
len save_T [55440]
[5.05000000e-07 1.18501210e-07 1.09167203e-02 4.54162497e-06
 4.32063724e+01 1.13421880e+00 6.95508793e+01]
len save_T [55440]
[5.05000000e-07 1.75991917e-05 1.98133826e-03 2.13661821e-05
 4.31263955e+01 1.24852747e+00 6.91596241e+01]
len save_T [55440]
[5.05000000e-07 1.75991917e-05 1.98133826e-03 2.13661821e-05
 4.31263955e+01 1.24852747e+00 6.91596241e+01]
len save_T [55440]
[5.05000000e-07 1.91641179e-04 9.27131488e-05 3.40265653e-07
 5.61790996e+01 1.94840836e+00 7.47446665e+01]
len save_T [55440]
[5.05000000e-07 1.91641179e-04 9.27131488e-05 3.40265653e-07
 5.61790996e+01 1.94840836e+00 

In [11]:
Nh = 4
Ll = 2
s = linspace(0,1,1000)
fhyp = Ll/2 * (tanh(2*Nh*s - Nh)/tanh(Nh) + 1)

figure('potato').clf()
plot(s,fhyp)